In [59]:
import numpy as np 
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Layer, Lambda, InputLayer
from tensorflow.keras.metrics import mean_squared_error as mse 
from tensorflow.keras import Model
from tensorflow.keras.losses import MeanSquaredError
import csv
import matplotlib.pyplot as plt 
fname = "./data/train_two_l50_j5_n15000_r0.csv"
data = np.genfromtxt(fname, delimiter=',', skip_header=2000)
np.random.shuffle(data)
test_data = data[-1000:]
np.savetxt("./data/test_data.csv", test_data, delimiter=",")

In [60]:
data.shape

(10517, 54)

In [61]:
data[0]

array([0.390625  , 0.1953125 , 0.        , 0.1953125 , 0.        ,
       0.        , 0.        , 0.1953125 , 0.78125   , 0.        ,
       0.1953125 , 0.390625  , 0.1953125 , 0.        , 0.        ,
       0.        , 0.        , 0.390625  , 0.        , 0.        ,
       0.        , 0.390625  , 0.        , 0.        , 0.        ,
       0.        , 0.78125   , 0.        , 0.        , 0.        ,
       0.390625  , 0.1953125 , 0.390625  , 0.        , 0.        ,
       0.1953125 , 0.        , 0.1953125 , 0.390625  , 0.1953125 ,
       0.390625  , 0.        , 0.1953125 , 0.5859375 , 0.1953125 ,
       0.1953125 , 0.390625  , 0.5859375 , 0.1953125 , 0.        ,
       0.216774  , 0.14191499, 0.1640625 , 0.17578125])

dataset structure : bm[50]|n_truth_prev[1]|n_truth[1]

In [62]:
def permute_row(row):
    truths = row[-2:]
    bm = row[:-2]
    permutated_bm = np.random.permutation(bm)
    permutated_row = np.append(permutated_bm, truths)
    return permutated_row
permute_row(data[0])

array([0.        , 0.        , 0.390625  , 0.390625  , 0.390625  ,
       0.        , 0.1953125 , 0.        , 0.        , 0.1953125 ,
       0.390625  , 0.1953125 , 0.1953125 , 0.216774  , 0.        ,
       0.        , 0.        , 0.1953125 , 0.1953125 , 0.        ,
       0.390625  , 0.390625  , 0.        , 0.        , 0.1953125 ,
       0.        , 0.1953125 , 0.78125   , 0.        , 0.1953125 ,
       0.1953125 , 0.        , 0.        , 0.390625  , 0.        ,
       0.14191499, 0.        , 0.        , 0.1953125 , 0.390625  ,
       0.        , 0.1953125 , 0.        , 0.5859375 , 0.390625  ,
       0.1953125 , 0.78125   , 0.        , 0.5859375 , 0.        ,
       0.        , 0.        , 0.1640625 , 0.17578125])

In [63]:
def augment_dataset(train_data, num_permutations=4):
    # Adds num_permutations to a sample (num_permutations+1)X the dataset
    fname = f"./data/augment_p{num_permutations}.csv"
    augmented_data = []
    for row in train_data:
        augmented_data.append(row)
        for i in range(num_permutations):
            permuted_row = permute_row(row)
            augmented_data.append(permuted_row)
    np.savetxt(fname, augmented_data, delimiter=",")
    return np.array(augmented_data)
augment_dataset(data)


array([[0.390625  , 0.1953125 , 0.        , ..., 0.14191499, 0.1640625 ,
        0.17578125],
       [0.        , 0.        , 0.        , ..., 0.5859375 , 0.1640625 ,
        0.17578125],
       [0.5859375 , 0.390625  , 0.1953125 , ..., 0.        , 0.1640625 ,
        0.17578125],
       ...,
       [0.390625  , 0.1953125 , 0.        , ..., 0.390625  , 0.37109375,
        0.375     ],
       [0.        , 0.1953125 , 0.390625  , ..., 0.390625  , 0.37109375,
        0.375     ],
       [0.1953125 , 0.1953125 , 0.390625  , ..., 0.        , 0.37109375,
        0.375     ]])

In [56]:
augmented_data = np.genfromtxt('./data/augment_p4.csv', delimiter=',')
augmented_data.shape

(52585, 54)